In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
img_dir_path = 'D:/Machine_Learning_DATA_Project/Deep Globe/reshaped_1024/train/images/'
mask_dir_path = 'D:/Machine_Learning_DATA_Project/Deep Globe/reshaped_1024/train/masks/'
outfile = 'D:/Machine_Learning_DATA_Project/Deep Globe/reshaped_1024/train/tfrecord/deepglobe_train.tfrecord'
img_names = sorted(os.listdir(img_dir_path))
mask_names = sorted(os.listdir(mask_dir_path))

In [3]:
def create_example(img_path,mask_path):
    img = tf.io.read_file(img_dir_path + img_path)
    decode_img = tf.io.decode_jpeg(img,channels=3)
    encode_img = tf.io.encode_jpeg(decode_img)
    
    mask = tf.io.read_file(mask_dir_path + mask_path)
    decode_mask = tf.io.decode_jpeg(mask,channels=3)
    encode_mask = tf.io.encode_jpeg(decode_mask)
    
    feature = {
        'image' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[encode_img.numpy()])),
        'mask' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[encode_mask.numpy()]))
    }
    
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example
    

In [5]:
def create_tfrecord(outfile,dataset):
    with tf.io.TFRecordWriter(outfile) as writer:
        for img_path,mask_path in tqdm(dataset,desc='writing_tfrecord',unit=' examples'):
            
            tf_example = create_example(img_path,mask_path)
            writer.write(tf_example.SerializeToString())

In [6]:
zipped = zip(img_names,mask_names)

In [7]:
create_tfrecord(outfile,zipped)

writing_tfrecord: 12848 examples [03:55, 54.65 examples/s]


In [ ]:
def parsh(example_proto):
    feature_keys = {
        'image' : tf.io.FixedLenFeature([],tf.string),
        'mask' : tf.io.FixedLenFeature([],tf.string)
    }
    
    example = tf.io.parse_single_example(example_proto, feature_keys)
    
    img = tf.io.decode_image(example['image'],channels=3)
    img = tf.reshape(img,(256,256,3))
    
    mask = tf.io.decode_image(example['mask'],channels=3)
    mask = tf.reshape(mask,(256,256,3))
    return img,mask

In [ ]:
train_file = outfile
dataset = tf.data.TFRecordDataset(train_file)
dataset = dataset.map(parsh)
batch_size=1
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)